# Emerging Technologies Assessment

## Introduction

This notebook explores the difference between classical and quantum algorithms through the
Deutsch and Deutsch–Jozsa problems. These problems demonstrate how quantum computation can
provide exponential speedups over classical deterministic algorithms for certain tasks.

In particular, the task is to determine whether a Boolean function is constant or balanced.
This may require evaluating all possible inputs while quantum algorithms can solve the problem with a single oracle query.

The notebook is structured into five problems, each building on the previous to progress
from classical computation to quantum implementations using Qiskit.

## Problem 1: Generating Random Boolean Functions

The Deutsch–Jozsa algorithm operates on Boolean functions that satisfy a *promise*:
the function is either **constant**, returning the same value for all inputs, or
**balanced**, returning `True` for exactly half of all possible inputs.

For a function of four Boolean inputs, there are \(2^4 = 16\) possible input
combinations. A balanced function must therefore return `True` for exactly 8 of
these inputs and `False` for the remaining 8.

The goal of this problem is to construct a Python function that randomly selects
and returns a Boolean function satisfying this promise.



In [1]:
import random



In [ ]:
#     Need a function to generate random_constant_balanced
#     Returns a randomly chosen Boolean function with four inputs.
#     Need an else statement
#     The function is guaranteed to be either constant or balanced.
#     Test cell to test_function. Something like test_function = random_constant_balanced() with prints

# All possible inputs for four Boolean variables (no itertools. Not in requirements.txt repo)
INPUTS = []
for a in [False, True]:
    for b in [False, True]:
        for c in [False, True]:
            for d in [False, True]:
                INPUTS.append((a, b, c, d))

def constant_function(value):
    """
    Returns a constant Boolean function of four Boolean inputs.
    """
    def f(a, b, c, d):
        return value
    return f

def balanced_function():
    """
    Returns a balanced Boolean function of four Boolean inputs.
    Exactly half of all possible inputs evaluate to True.
    """
    true_inputs = set(random.sample(INPUTS, k=len(INPUTS) // 2))

    def f(a, b, c, d):
        return (a, b, c, d) in true_inputs

    return f



In [4]:


def random_constant_balanced():
    """
    Returns a randomly chosen Boolean function of four Boolean inputs.
    The function is guaranteed to be either constant or balanced.
    """
    if random.choice([True, False]):
        value = random.choice([False, True])
        f = constant_function(value)
        f.function_type = "constant"
        return f
    else:
        f = balanced_function()
        f.function_type = "balanced"
        return f


### Verification of Balanced Function

To verify that the generated function is balanced, the function is evaluated on
all 2⁴ = 16 possible Boolean input combinations. The number of True and False
outputs is then counted.

A function is considered balanced if exactly half of the outputs evaluate to
True and the remaining half evaluate to False. The counts produced below
demonstrate that the balanced function generator satisfies this condition.

In [3]:
f_bal = balanced_function()
outputs = [f_bal(*x) for x in INPUTS]

sum(outputs), len(outputs) - sum(outputs)


(8, 8)

## Problem 2: Classical Testing for Function Type

In [5]:
# Code cell

## Problem 3: Quantum Oracles

In [6]:
# Code cell

## Problem 4: Deutsch's Algorithm with Qiskit


In [7]:
# Code cell

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm

In [8]:
# Code cell